# 6. gyakorlat

In [1]:
from pyspark import SparkConf, SparkContext 

conf = SparkConf() 
sc = SparkContext(conf=conf)

In [2]:
text = sc.textFile('alkotmany.txt')

## 0. feladat

WordCount kicsit másképp: `groupByKey` egy szekvenciába összegyűjti az azonos kulcsú elemeket. `mapValues` függvény úgy alakítja át az RDD-t, hogy a kulcsokat változatlanul hagyja, csak az értékeken végez átalakítást.

In [13]:
text\
    .flatMap(lambda l: l.lower().replace(':',"").replace(",","").replace(".", "").replace("!","").split(" "))\
    .map(lambda w: (w.lower(), 1))\
    .groupByKey()\
    .mapValues(sum)\
    .sortBy(lambda t: t[1], False)\
    .take(5)

[('a', 41), ('és', 24), ('az', 19), ('hogy', 17), ('valljuk', 9)]

# 1. Milyen hosszú a leghosszabb szó?

Fold olyan minta a reduce csak kezdőértékét is vár (reducenál ez automatikusan a legelső elem)

In [14]:
text\
    .flatMap(lambda line: line.lower().replace(':',"").replace(",","").replace(".", "").replace("!","").split(" "))\
    .map(lambda word: len(word))\
    .fold(0, max)

18

Egy kicsit ügyesebben (`key` paraméter adja meg az maximumkiválasztásnál az adott érték milyen kulccsal szerepeljen):

In [15]:
len(text\
    .flatMap(lambda line: line.lower().replace(':',"").replace(",","").replace(".", "").replace("!","").split(" "))\
    .max(key=lambda word: len(word)))

18

Lehetséges megoldás még, hogy rendezzük az RDD-t és vesszük az első elemét.

## 2. Melyik a leghosszabb szó és milyen hosszú?

In [17]:
text\
    .flatMap(lambda line: line.lower().replace(':',"").replace(",","").replace(".", "").replace("!","").split(" "))\
    .map(lambda word: (word, len(word)))\
    .max(key=lambda word: word[1])

('nemzetiszocialista', 18)

Nem muszáj mindent rdd-n belül megoldani.

In [18]:
maxlength = text\
    .flatMap(lambda line: line.lower().replace(':',"").replace(",","").replace(".", "").replace("!","").split(" "))\
    .max(key=lambda word: len(word))

print(maxlength, len(maxlength))

nemzetiszocialista 18


## 3. WordMean

In [29]:
text\
    .flatMap(lambda line: line.split(" "))\
    .map(lambda word: len(word))\
    .mean()

6.974947807933194

## 4. Adjuk meg kezdőbetűnként a leghosszabb szó hosszát

Minden szóhoz egy párt rendelünk ami a szó első szava és a szó hossza. Minden különböző kulcs esetén kiválasztjuk a maximális értékét. Majd rendezzük kulcs szerint (str lexikografikus rendezést eredményez).

In [27]:
text.flatMap(lambda line: line.lower().replace(':',"").replace(",","").replace(".", "").replace("!","").split(" "))\
    .map(lambda word: (word[0], len(word)))\
    .reduceByKey(max)\
    .sortByKey(keyfunc=str)\
    .collect()

[('t', 17),
 ('ú', 2),
 ('b', 13),
 ('o', 12),
 ('g', 12),
 ('í', 7),
 ('l', 14),
 ('p', 9),
 ('j', 10),
 ('c', 6),
 ('ü', 8),
 ('d', 12),
 ('z', 8),
 ('m', 15),
 ('a', 15),
 ('n', 18),
 ('é', 16),
 ('k', 14),
 ('f', 16),
 ('v', 12),
 ('h', 14),
 ('s', 15),
 ('i', 8),
 ('e', 17),
 ('á', 11),
 ('r', 12),
 ('ő', 8),
 ('ö', 17),
 ('u', 11),
 ('1', 7)]

## 5. Pí becslés

In [33]:
from random import random

num_of_points = 1000000

points = sc\
    .parallelize([1 for i in range(num_of_points)])\
    .filter(lambda _: (random() ** 2 + random() ** 2) < 1)

4 * points.count() / num_of_points

3.140492

# 6. feladat maxTemp

In [34]:
data = sc.textFile("tempBudapestMeteoBlue.csv")\
    .map(lambda t: t.split(","))\
    .filter(lambda l: l[0].startswith('2020'))\
    .map(lambda t: (t[0].split("T"), t[1]))\
    .map(lambda t: (t[0][0], (t[0][1], t[1])))\
    .reduceByKey(lambda a, b: a if float(a[1]) > float(b[1]) else b)

data.collect()

[('20200914', ('1700', '30.651459')),
 ('20200915', ('1600', '32.06146')),
 ('20200916', ('1600', '31.431458')),
 ('20200917', ('1500', '30.111458')),
 ('20200918', ('1600', '22.861458')),
 ('20200919', ('1600', '23.311459')),
 ('20200920', ('1700', '25.321459')),
 ('20200922', ('1600', '27.831459')),
 ('20200923', ('1400', '27.131458')),
 ('20200924', ('1500', '26.421457')),
 ('20200926', ('0000', '17.621458')),
 ('20200927', ('1500', '18.401459')),
 ('20200921', ('1300', '27.171457')),
 ('20200925', ('1800', '22.981459')),
 ('20200928', ('1500', '19.731459'))]

## Accumulator
Hadoop counter-jéhez hasonló, olyan változó, amit csak növelni lehet, és azt is csak asszociatív műveletekkel. Az `accumulator` értéke automatikusan szinkronizálásra kerül az RDD kiértékelését követően. `Foreach` általános bejárás, egyetlen paramétere egy `T->None`-ba képező függvény.

In [35]:
w_count = sc.accumulator(0)

def count_word(s):
    if 'MAGYAR' in s.upper(): 
        w_count.add(1) # add accumulator növelése

text\
    .flatMap(lambda x : x.split(" "))\
    .foreach(count_word)

print(w_count)

12


## Broadcast
Egy read-only érték, amelyet minden gépen el tudunk érni.

In [37]:
w_count = sc.accumulator(0)
search_for = sc.broadcast(['MAGYAR'])

def count_word2(s):
    if search_for.value[0] in s.upper(): # value broadcast változó értéke
        w_count.add(1)

text\
    .flatMap(lambda x : x.split(" "))\
    .foreach(count_word2)

print(w_count)

12


## Maxtemp2

Oldjuk meg a MaxTemp feladatot úgy, hogy közben összeszámoljuk hány nap volt magasabb a hőmérséklet 30 foknál.

In [38]:
over_count = sc.accumulator(0)

def add_and_increment(t):
    if float(t[1][1]) > 30:
        over_count.add(1)
    return t

data = sc.textFile("tempBudapestMeteoBlue.csv")\
    .map(lambda t: t.split(","))\
    .filter(lambda l: l[0].startswith('2020'))\
    .map(lambda t: (t[0].split("T"), t[1]))\
    .map(lambda t: (t[0][0], (t[0][1], t[1])))\
    .reduceByKey(lambda a,b: a if float(a[1]) > float(b[1]) else b)\
    .map(add_and_increment)

print(data.collect())
print(over_count.value)

[('20200914', ('1700', '30.651459')), ('20200915', ('1600', '32.06146')), ('20200916', ('1600', '31.431458')), ('20200917', ('1500', '30.111458')), ('20200918', ('1600', '22.861458')), ('20200919', ('1600', '23.311459')), ('20200920', ('1700', '25.321459')), ('20200922', ('1600', '27.831459')), ('20200923', ('1400', '27.131458')), ('20200924', ('1500', '26.421457')), ('20200926', ('0000', '17.621458')), ('20200927', ('1500', '18.401459')), ('20200921', ('1300', '27.171457')), ('20200925', ('1800', '22.981459')), ('20200928', ('1500', '19.731459'))]
4
